##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aprendizado por transferência para áudio com o TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/audio_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
  <td>     <a href="https://tfhub.dev/google/yamnet/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a> </td>
</table>

Neste notebook do Colab, você verá como usar o [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) (criador de modelos do TF Lite) para treinar um modelo personalizado de classificação de áudio.

A biblioteca Model Maker usa aprendizado por transferência para simplificar o processo de treinar um modelo do TensorFlow Lite usando um dataset personalizado. Retreinar um modelo do TensorFlow Lite com seu próprio dataset personalizado reduz a quantidade necessária de dados de treinamento e tempo.

Este notebook faz parte do [Codelab para personalizar um modelo de áudio e implantar no Android](https://codelabs.developers.google.com/codelabs/tflite-audio-classification-custom-model-android).

Você usará um dataset personalizado de pássaros e exportará um modelo do TF Lite que pode ser usado em um celular, um modelo TensorFlow.JS que pode ser usado para inferência no navegador e também uma versão SavedModel que pode ser usada como serviço.


## Instale as dependências


In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

## Importe o TensorFlow, o Model Maker e outras bibliotecas

Dentre as dependências necessárias, você usará o TensorFlow e o Model Maker. Além delas, as outras são para manipulação e reprodução de áudio, além de visualização.

In [ ]:
import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import glob
import random

from IPython.display import Audio, Image
from scipy.io import wavfile

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## Dataset Birds

O dataset Birds é uma coleção educativa do canto de cinco pássaros:

- White-breasted Wood-Wren (Uirapuru-de-peito-branco)
- House Sparrow (Pardal-doméstico)
- Red Crossbill (Cruza-bico)
- Chestnut-crowned Antpitta (Grallaria ruficapilla)
- Azara's Spinetail (Synallaxis azarae)

O áudio original vem de [Xeno-canto](https://www.xeno-canto.org/), um site dedicado ao compartilhamento de cantos de pássaros de todo o mundo.

Vamos começar pelo download dos dados.

In [ ]:
birds_dataset_folder = tf.keras.utils.get_file('birds_dataset.zip',
                                                'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/birds_dataset.zip',
                                                cache_dir='./',
                                                cache_subdir='dataset',
                                                extract=True)
                                                

## Explore os dados

Os áudios já estão divididos em pastas de teste e treinamento. Dentro de cada pasta, há uma pasta para cada pássaro, usando `bird_code` como nome.

Todos os áudios são mono, com taxa de amostragem de 16 kHz.

Confira mais informações sobre cada arquivo no arquivo `metadata.csv`, que contém todos os autores, licenças e informações adicionais. Para este tutorial, não é preciso ler esse arquivo.

In [ ]:
# @title [Run this] Util functions and data structures.

data_dir = './dataset/small_birds_dataset'

bird_code_to_name = {
  'wbwwre1': 'White-breasted Wood-Wren',
  'houspa': 'House Sparrow',
  'redcro': 'Red Crossbill',  
  'chcant2': 'Chestnut-crowned Antpitta',
  'azaspi1': "Azara's Spinetail",   
}

birds_images = {
  'wbwwre1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg/640px-Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg', # 	Alejandro Bayer Tamayo from Armenia, Colombia 
  'houspa': 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/House_Sparrow%2C_England_-_May_09.jpg/571px-House_Sparrow%2C_England_-_May_09.jpg', # 	Diliff
  'redcro': 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Red_Crossbills_%28Male%29.jpg/640px-Red_Crossbills_%28Male%29.jpg', #  Elaine R. Wilson, www.naturespicsonline.com
  'chcant2': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Chestnut-crowned_antpitta_%2846933264335%29.jpg/640px-Chestnut-crowned_antpitta_%2846933264335%29.jpg', # 	Mike's Birds from Riverside, CA, US
  'azaspi1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/Synallaxis_azarae_76608368.jpg/640px-Synallaxis_azarae_76608368.jpg', # https://www.inaturalist.org/photos/76608368
}

test_files = os.path.abspath(os.path.join(data_dir, 'test/*/*.wav'))

def get_random_audio_file():
  test_list = glob.glob(test_files)
  random_audio_path = random.choice(test_list)
  return random_audio_path


def show_bird_data(audio_path):
  sample_rate, audio_data = wavfile.read(audio_path, 'rb')

  bird_code = audio_path.split('/')[-2]
  print(f'Bird name: {bird_code_to_name[bird_code]}')
  print(f'Bird code: {bird_code}')
  display(Image(birds_images[bird_code]))

  plttitle = f'{bird_code_to_name[bird_code]} ({bird_code})'
  plt.title(plttitle)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

print('functions and data structures created')

### Reproduza alguns áudios

Para entender melhor os dados, vamos ouvir arquivos de áudio aleatórios do dataset de teste.

Observação: posteriormente neste notebook, você executará a inferência nestes áudios para teste.

In [ ]:
random_audio = get_random_audio_file()
show_bird_data(random_audio)

## Treine o modelo

Ao usar o Model Maker para áudio, você precisa começar pela especificação do modelo, que será o modelo base do qual o novo modelo extrairá informações para aprender sobre as novas classes. Além disso, ele afeta como o dataset será transformado para respeitar os parâmetros de especificação dos modelos, como taxa de amostragem e número de canais.

[YAMNet](https://tfhub.dev/google/yamnet/1) é um classificador de eventos de áudio treinado com o dataset AudioSet para prever eventos de áudios da ontologia AudioSet.

É esperado que a entrada esteja em 16 kHz e tenha 1 canal.

Você não precisa fazer novas amostragens, pois o Model Maker cuida dessa tarefa.

- `frame_length` decide o tamanho de cada amostra de treinamento. Neste caso, EXPECTED_WAVEFORM_LENGTH * 3s

- `frame_steps` indica a distância entre as amostras de treinamento. Neste caso, a amostra i começará EXPECTED_WAVEFORM_LENGTH * 6s após a amostra (i-1).

O motivo de definir esses valores é poder contornar algumas limitações de datasets do mundo real.

Por exemplo: no dataset Birds, os pássaros não cantam o tempo todo. Eles cantam, descansam, cantam novamente, e há ruídos nesses períodos. Usar um quadro longo ajuda a capturar o canto, mas, se for longo demais, reduz o número de amostras de treinamento.


In [ ]:
spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=3 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=6 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

## Carregue os dados

O Model Maker tem uma API para carregar os dados a partir de uma pasta e colocá-los no formato esperado pela especificação do modelo.

Os datasets de treinamento e teste são baseados nas pastas. O dataset de validação será criado como 20% do de treinamento.

Observação: é importante usar `cache=True` para deixar o treinamento posterior mais rápido, mas isso exigirá mais RAM para armazenar os dados. Para o dataset Birds, não é um problema, já que são apenas 300 MB, mas, se você usar seus próprios dados, tenha cuidado com isso.


In [ ]:
train_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'train'), cache=True)
train_data, validation_data = train_data.split(0.8)
test_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'test'), cache=True)

## Treine o modelo

O classificador de áudio audio_classifier conta com o método [`create`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create), que cria um modelo e já começa a treiná-lo.

Você pode personalizar diversos parâmetros. Confira mais informações na documentação.

Neste primeiro teste, você usará as configurações padrão e fará o treinamento com 100 épocas.

Observação: a primeira época leva mais tempo do que todas as outras porque é quando o cache é criado. Depois disso, cada época leva aproximadamente 1 segundo.

In [ ]:
batch_size = 128
epochs = 100

print('Training the model')
model = audio_classifier.create(
    train_data,
    spec,
    validation_data,
    batch_size=batch_size,
    epochs=epochs)

A exatidão está boa, mas é importante executar o passo de avaliação com os dados de teste para verificar se o modelo tem bons resultados com dados independentes.

In [ ]:
print('Evaluating the model')
model.evaluate(test_data)

## Sobre o modelo

Ao treinar um classificador, é útil conferir a [matriz de confusão](https://en.wikipedia.org/wiki/Confusion_matrix), que fornece detalhes sobre o desempenho do classificador para os dados de teste.

O Model Maker já cria a matriz de confusão para você.

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = test_labels
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## Teste o modelo (opcional)

Você pode testar o modelo com uma amostra de áudio do dataset de teste apenas para ver os resultados.

Primeiro, obtenha o modelo de serviço.

In [ ]:
serving_model = model.create_serving_model()

print(f'Model\'s input shape and type: {serving_model.inputs}')
print(f'Model\'s output shape and type: {serving_model.outputs}')

Use o áudio aleatório que você carregou anteriormente.

In [ ]:
# if you want to try another file just uncoment the line below
random_audio = get_random_audio_file()
show_bird_data(random_audio)

O modelo criado tem uma janela de entrada fixa.

Para um determinado arquivo de áudio, você precisará dividi-lo em janelas de dados do tamanho esperado. Talvez a última janela precise ser preenchida com zeros.

In [ ]:
sample_rate, audio_data = wavfile.read(random_audio, 'rb')

audio_data = np.array(audio_data) / tf.int16.max
input_size = serving_model.input_shape[1]

splitted_audio_data = tf.signal.frame(audio_data, input_size, input_size, pad_end=True, pad_value=0)

print(f'Test audio path: {random_audio}')
print(f'Original size of the audio data: {len(audio_data)}')
print(f'Number of windows for inference: {len(splitted_audio_data)}')

Você fará um loop percorrendo todos os áudios divididos e aplicará o modelo em cada um deles.

O modelo que você acabou de treinar tem duas saídas: a saída original do YAMNet e a que você treinou. Isso é importante, pois ambientes reais são mais complicados do que sons de pássaros. Você pode usar a saída do YAMNet para filtrar os áudios não relevantes. Por exemplo: para o dataset Birds, se o YAMNet não estiver classificando pássaros ou animais, isso pode mostrar que a saída do seu modelo pode ter uma classificação irrelevante.

Veja abaixo as duas saídas para entender melhor a relação entre elas. A maioria dos erros que seu modelo comete são quando a previsão do YAMNet não estará relacionada à sua área (como pássaros).

In [ ]:
print(random_audio)

results = []
print('Result of the window ith:  your model class -> score,  (spec class -> score)')
for i, data in enumerate(splitted_audio_data):
  yamnet_output, inference = serving_model(data)
  results.append(inference[0].numpy())
  result_index = tf.argmax(inference[0])
  spec_result_index = tf.argmax(yamnet_output[0])
  t = spec._yamnet_labels()[spec_result_index]
  result_str = f'Result of the window {i}: ' \
  f'\t{test_data.index_to_label[result_index]} -> {inference[0][result_index].numpy():.3f}, ' \
  f'\t({spec._yamnet_labels()[spec_result_index]} -> {yamnet_output[0][spec_result_index]:.3f})'
  print(result_str)


results_np = np.array(results)
mean_results = results_np.mean(axis=0)
result_index = mean_results.argmax()
print(f'Mean result: {test_data.index_to_label[result_index]} -> {mean_results[result_index]}')

## Exporte o modelo

A última etapa é exportar o modelo que será usado em dispositivos embarcados ou no navegador.

O método `export` exporta os dois formatos.

In [ ]:
models_path = './birds_models'
print(f'Exporing the TFLite model to {models_path}')

model.export(models_path, tflite_filename='my_birds_model.tflite')

Além disso, você pode exportar a versão SavedModel para uso em serviço ou em um ambiente Python.

In [ ]:
model.export(models_path, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## Próximos passos

Você conseguiu.

Agora, seu modelo pode ser implantado em dispositivos móveis usando a [API de tarefas AudioClassifier do TF Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier).

Além disso, você pode fazer o mesmo processo com seus próprios dados e classes diferentes. Confira a documentação do [Model Maker para classificação de áudio](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier).

Para saber mais, confira os aplicativos completos de referência para [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/) e [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios).